<a href="https://colab.research.google.com/github/MevrouwHelderder/Assignments/blob/main/Final_Assignment_Energy_production_and_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final assignment - Energy production and consumption



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext google.colab.data_table

In [2]:
path_base = "https://raw.githubusercontent.com/MevrouwHelderder/final_assignment/main/"

paths = {
    "energy_total" : path_base + "owid-energy-data.csv",
    "net_zero_pledge" : path_base + "net-zero-targets.csv",
    "agri_land_use_total" : path_base + "total-agricultural-area-over-the-long-term.csv",
    "grazing" : path_base + "grazing-land-use-over-the-long-term.csv",
    "cropland" : path_base + "cropland-use-over-the-long-term.csv"
}

dataframes = {}

for key, value in paths.items():
  dataframes[key] = pd.read_csv(value)

energy_total_df = dataframes["energy_total"]
# net_zero_pledge_df = dataframes["net_zero_pledge"]
# agri_land_use_total_df = dataframes["agri_land_use_total"]
# grazing_df = dataframes["grazing"]
# cropland_df = dataframes["cropland"]

# Who changed the best and worse over the past x years (50?)

In [13]:
# Renaming column names so there is more uniformity
for column in energy_total_df.columns:
    if "_energy_per_capita" in column:
        new_name = column.replace("_energy_per_capita", "_cons_per_capita")
        energy_total_df.rename(columns={column: new_name}, inplace=True)
    if "renewable" in column:
        new_name = column.replace("renewable_", "renewables_")
        energy_total_df.rename(columns={column: new_name}, inplace=True)


In [12]:
# Setting the index
energy_total_df = energy_total_df.set_index("year")
energy_total_df.index = pd.to_datetime(energy_total_df.index, format="%Y", errors="coerce")

# Renaming some columns that won't fit the function later
energy_total_df.rename(columns={"other_renewables_elec_per_capita_exc_biofuel" :"other_renewables_exc_biofuel_elec_per_capita",
                             "other_renewables_share_elec_exc_biofuel": "other_renewables_exc_biofuel_share_elec"}, inplace=True)

# Exploration
The big dataset contains data of several categories, including columns for each categorie and columns for all items in each categorie. 

I did some checks to verify what columns store combinations of other columns by adding the, presumed, individual measurements up and comparing the result to the, presumed, combination column.


In [27]:
# Making a test dataframe to check what the columns store
test = energy_total_df[energy_total_df["country"]=="Germany"].iloc[100: 119,:]
# make a column for all fossil fuels: 
test['fossil_electricity'] = (test['coal_electricity'] + test[ 'oil_electricity'] + test['gas_electricity']).round(3)

In [40]:
renew_test = test.loc[: ,['country', 
          'wind_electricity', 
          'solar_electricity', 
          'hydro_electricity',
          'biofuel_electricity',
          "other_renewables_exc_biofuel_electricity",
          "renewables_electricity",]].copy()


renew_test["wind_solar_hydro_bio_other"] = (renew_test["wind_electricity"] + renew_test["solar_electricity"] + renew_test["hydro_electricity"] + renew_test["biofuel_electricity"] + renew_test["other_renewables_exc_biofuel_electricity"]).round(3)
renew_test

,country,wind_electricity,solar_electricity,hydro_electricity,biofuel_electricity,other_renewables_exc_biofuel_electricity,renewables_electricity,wind_solar_hydro_bio_other
year,,,,,,,,
2000-01-01,Germany,9.35,0.06,21.73,4.33,0.00,35.47,35.47
2001-01-01,Germany,10.46,0.12,22.73,4.59,0.00,37.90,37.90
2002-01-01,Germany,15.86,0.19,23.12,5.31,0.00,44.48,44.48
2003-01-01,Germany,19.09,0.31,18.32,8.95,0.00,46.67,46.67
2004-01-01,Germany,26.02,0.56,20.75,10.64,0.00,57.97,57.97
2005-01-01,Germany,27.77,1.28,19.64,14.71,0.00,63.40,63.40
2006-01-01,Germany,31.32,2.22,20.03,18.94,0.00,72.51,72.51
2007-01-01,Germany,40.51,3.08,21.17,24.62,0.00,89.38,89.38
2008-01-01,Germany,41.38,4.42,20.44,28.02,0.02,94.28,94.28


In [51]:
presumed_renewables = (test["wind_electricity"] + test["solar_electricity"] + test["hydro_electricity"] + test["biofuel_electricity"] + test["other_renewables_exc_biofuel_electricity"]).round(3)
presumed_low_carbon = (test['nuclear_electricity'] + test['renewables_electricity']).round(3)
presumed_all = (test['low_carbon_electricity'] + test['renewables_electricity']).round(3)

In [49]:
presumed_low_carbon == test['low_carbon_electricity']

year
2000-01-01    True
2001-01-01    True
2002-01-01    True
2003-01-01    True
2004-01-01    True
2005-01-01    True
2006-01-01    True
2007-01-01    True
2008-01-01    True
2009-01-01    True
2010-01-01    True
2011-01-01    True
2012-01-01    True
2013-01-01    True
2014-01-01    True
2015-01-01    True
2016-01-01    True
2017-01-01    True
2018-01-01    True
dtype: bool

In [50]:
presumed_renewables == test['renewables_electricity']


year
2000-01-01    True
2001-01-01    True
2002-01-01    True
2003-01-01    True
2004-01-01    True
2005-01-01    True
2006-01-01    True
2007-01-01    True
2008-01-01    True
2009-01-01    True
2010-01-01    True
2011-01-01    True
2012-01-01    True
2013-01-01    True
2014-01-01    True
2015-01-01    True
2016-01-01    True
2017-01-01    True
2018-01-01    True
dtype: bool

In [37]:
all_test = test.loc[: ,['country', 
          'low_carbon_electricity', 
          'fossil_electricity', 
          'electricity_generation']].copy()

all_test['all'] = (all_test['low_carbon_electricity'] + all_test['fossil_electricity']).round(3)
all_test

,country,low_carbon_electricity,fossil_electricity,electricity_generation,all
year,,,,,
2000-01-01,Germany,205.08,367.22,572.30,572.30
2001-01-01,Germany,209.20,372.69,581.89,581.89
2002-01-01,Germany,209.32,372.64,581.96,581.96
2003-01-01,Germany,211.73,390.81,602.54,602.54
2004-01-01,Germany,225.04,385.24,610.28,610.28
2005-01-01,Germany,226.45,386.96,613.41,613.41
2006-01-01,Germany,239.78,390.03,629.81,629.81
2007-01-01,Germany,229.91,402.40,632.31,632.31
2008-01-01,Germany,242.77,390.43,633.20,633.20


Conclusion: 
* **wind, solar, hydro, biofuel, other renewables exc biofuel, nuclear, coal, oil and gas** are exclusive energy sources

* **renewables** is the sum of wind, solar, hydro, biofuel, other renewables exc biofuel
* **low_carbon** is the sum of wind, solar, hydro, biofuel, other renewables exc biofuel and nuclear
* ** fossil** could be the sum of coal, oil and gas


In [6]:
# Renewables:
renew_list = ["wind", "solar", "hydro", "biofuel"]

# Fossil:
fossil_list = ["coal", "oil", "gas"]

# low_carbon:
low_carbon_list = ["low_carbon", "nuclear", "other_renewables", "renewables"]

# base column name format: 
base_list = ["_electricity", "_elec_per_capita", "_consumption", "_cons_per_capita", "_cons_change_pct", "_cons_change_twh"]

# columns that won't work with the function: 
manual = ['other_renewables_exc_biofuel_electricity', 'other_renewables_exc_biofuel_elec_per_capita']


In [7]:
# function to create a new list from a list of topics and a list of measurements
# goal: select the needed columns from the big database without having to type them out individually

def combine_lists(topic_list, measurement_list):
    new_columns = ["country", "iso_code", "population", "gdp"]
    for topic_item in topic_list:
        for measurement_item in measurement_list:
            new_columns.append(topic_item + measurement_item)
    return new_columns


In [8]:
# creating the column names
renewables_columns = (combine_lists(renew_list, base_list)) + manual

In [9]:
renewables_df = total_energy.loc[:,renewables_columns]

In [10]:
renewables_df

,country,iso_code,population,gdp,wind_electricity,wind_elec_per_capita,wind_consumption,wind_cons_per_capita,wind_cons_change_pct,wind_cons_change_twh,...,other_renewables_cons_change_pct,other_renewables_cons_change_twh,renewables_electricity,renewables_elec_per_capita,renewables_consumption,renewables_cons_per_capita,renewables_cons_change_pct,renewables_cons_change_twh,other_renewables_exc_biofuel_electricity,other_renewables_exc_biofuel_elec_per_capita
year,,,,,,,,,,,,,,,,,,,,,
1900-01-01,Afghanistan,AFG,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1901-01-01,Afghanistan,AFG,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902-01-01,Afghanistan,AFG,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903-01-01,Afghanistan,AFG,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904-01-01,Afghanistan,AFG,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-01,Zimbabwe,ZWE,14751101.0,2.194784e+10,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,4.30,291.504,NaN,NaN,NaN,NaN,0.0,0.0
2018-01-01,Zimbabwe,ZWE,15052191.0,2.271535e+10,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,5.46,362.738,NaN,NaN,NaN,NaN,0.0,0.0
2019-01-01,Zimbabwe,ZWE,15354606.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,4.58,298.282,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
sum

Renewable energy is energy derived from natural sources that are replenished at a higher rate than they are consumed. 

"country", "year", "iso_code", "population", "gdp", "electricity_demand", 


**electricity_demand**. Electricity demand, measured in terawatt-hours
**electricity_generation**. Electricity generation, measured in terawatt-hours
**energy_per_capita**. Primary energy consumption per capita, measured in kilowatt-hours
**energy_cons_change_pct**. Annual percentage change in primary energy consumption
**energy_cons_change_twh**. Annual change in primary energy consumption, measured in terawatt-hours

**fossil_electricity**. 	Electricity generation from fossil fuels, measured in terawatt-hours. (Sum of coal, oil and gas.)
**fossil_elec_per_capita**. 	Per capita electricity generation from fossil fuels, measured in kilowatt-hours. 
**fossil_fuel_consumption**. 	Fossil fuel consumption, measured in terawatt-hours. 
**fossil_energy_per_capita**. Per capita fossil fuel consumption, measured in kilowatt-hours. 
**fossil_cons_change_pct**. 	Annual percentage change in fossil fuel consumption
**fossil_cons_change_twh**. 	Annual change in fossil fuel consumption, measured in terawatt-hours

**biofuel_electricity**. 	Electricity generation from biofuels, measured in terawatt-hours
**biofuel_elec_per_capita**. 	Per capita electricity generation from biofuels, measured in kilowatt-hours
**biofuel_consumption**. 	Primary energy consumption from biofuels
**biofuel_cons_per_capita**. 	Per capita primary energy consumption from biofuels
**biofuel_cons_change_pct**. 	Annual percentage change in biofuel consumption
**biofuel_cons_change_twh**. 	Annual change in biofuel consumption

**coal_electricity**. 		Electricity generation from coal
**coal_elec_per_capita**. 	Per capita electricity generation from coal
**coal_consumption**. 		Primary energy consumption from coal
**coal_cons_per_capita**. 	Per capita primary energy consumption from coal
**coal_cons_change_twh**. 	Annual change in coal consumption
**coal_cons_change_pct**. 	Annual percentage change in coal consumption

**gas_electricity**. 		Electricity generation from gas
**gas_elec_per_capita**. 	Per capita electricity generation from gas
**gas_consumption**. 		Primary energy consumption from gas
**gas_energy_per_capita**. 	Per capita primary energy consumption from gas
**gas_cons_change_twh**. 	Annual change in gas consumption
**gas_cons_change_pct**.  	Annual percentage change in gas consumption

**hydro_electricity**. 		Electricity generation from hydropower
**hydro_elec_per_capita**.	Per capita electricity generation from hydropower
**hydro_consumption**.		Primary energy consumption from hydropower
**hydro_energy_per_capita**.	Per capita primary energy consumption from hydropower
**hydro_cons_change_pct**.	Annual percentage change in hydropower consumption 
**hydro_cons_change_twh**.	Annual change in hydropower consumption

**oil_electricity**.		Electricity generation from oil
**oil_elec_per_capita**.	Per capita electricity generation from oil
**oil_consumption**.		Primary energy consumption from oil
**oil_energy_per_capita**.	Per capita primary energy consumption from oil
**oil_cons_change_twh**.	Annual change in oil consumption
**oil_cons_change_pct**.	Annual percentage change in oil consumption

**low_carbon_electricity**.	Electricity generation from low-carbon sources, measured in terawatt-hours. (Sum of renewables and nuclear.)
**low_carbon_elec_per_capita**.Per capita electricity generation from low-carbon sources, measured in kilowatt-hours
**low_carbon_consumption**.	Primary energy consumption from low-carbon sources, measured in terawatt-hours
**low_carbon_energy_per_capita**.Per capita primary energy consumption from low-carbon sources
**low_carbon_cons_change_twh**.Annual change in low-carbon energy consumption, measured in terawatt-hours
**low_carbon_cons_change_pct**.Annual percentage change in low-carbon energy consumption

**nuclear_electricity**.	Electricity generation from nuclear power
**nuclear_elec_per_capita**.	Per capita electricity generation from nuclear power
**nuclear_consumption**.	Primary energy consumption from nuclear power 
**nuclear_energy_per_capita**.Per capita primary energy consumption from nuclear
**nuclear_cons_change_twh**.	Annual change in nuclear consumption
**nuclear_cons_change_pct**.	Annual percentage change in nuclear consumption

**other_renewable_electricity**.Electricity generation from other renewables including biofuels
**other_renewable_elec_per_capita**.Per capita electricity generation from other renewables including biofuels. 
**other_renewable_consumption**.Primary energy consumption from other renewables including biofuels
**other_renewable_energy_per_capita**.Per capita primary energy consumption from other renewables including biofuels
**other_renewable_cons_change_twh**.Annual change in other renewables consumption
**other_renewable_cons_change_pct**.Annual percentage change in other renewables consumption

**renewable_electricity**.	Electricity generation from renewables including biofuels
**renewable_elec_per_capita**.Per capita electricity generation from renewables including biofuels. 
**renewable_consumption**.	Primary energy consumption from renewables including biofuels
**renewable_energy_per_capita**.Per capita primary energy consumption from renewables including biofuels
**renewable_cons_change_twh**.Annual change in renewables consumption
**renewable_cons_change_pct**.Annual percentage change in renewables consumption

**solar_electricity**.		Electricity generation from solar 
**solar_elec_per_capita**.	Per capita electricity generation from solar
**solar_consumption**.		rimary energy consumption from solar
**solar_energy_per_capita**.	er capita primary energy consumption from solar
**solar_cons_change_twh**.	Annual change in  solar consumption
**solar_cons_change_pct**.	Annual percentage change in  solar consumption

**wind_electricity**.		Electricity generation from wind
**wind_elec_per_capita**.	Per capita electricity generation from wind
**wind_consumption**.		rimary energy consumption from wind
**wind_energy_per_capita**.	Per capita primary energy consumption from wind
**wind_cons_change_twh**.	Annual change in wind consumption
**wind_cons_change_pct**.	Annual percentage change in wind consumption

**carbon_intensity_elec**. 	Carbon intensity of electricity production, measured in grams of carbon dioxide emitted per kilowatt-hour

# What are the safest and cleanest?

https://ourworldindata.org/safest-sources-of-energy#the-safest-energy-sources-are-also-the-cleanest

This is your final assignment. You'll get a lot of freedom in doing this assignment but that also means you have to make choices and explain the reasoning behind those choices in your report.

For this assignment you can use any dataset you can find from the [Our World in Data website](https://ourworldindata.org/).

Please formulate an answer to the following three questions in your report.

* **What is the biggest predictor of a large CO2 output per capita of a country?**
* **which countries are making the biggest strides in decreasing CO2 output?**
* **which non-fossil fuel energy technology will have the best price in the future?**




---


**1: Biggest predictor of CO2 output**

To determine this you may want to consider things like GDP per capita, diets, number of cars per capita, various energy source, mobility and other factors.

Your answer can also be a specific combination of certain factors.


---


**2: Biggest strides in decreasing CO2 output**

You'll need to find the relative CO2 output for each country to be able to calculate this. But countries can have growing and shrinking populations too, so it's probably a good idea to take this into account as well.


---


**3: Best future price for non-fossil fuel energy**

To be able to predict prices you'll probably need to use linear regression over the various non-fossil fuel options.


---


**Submitting your Assignment**

Once you're done with this module, you can go to the next item where you'll be able to submit your assignment.

Please submit both your written report and all notebooks you've created in creating the report. Make sure everything works before submitting.



# Thoughts: 
**For the biggest predictor of a large CO2 output per capita of a country, you can use the following datasets:**

* CO2 emissions by country: This dataset provides historical CO2 emissions data for different countries over time.
* Energy consumption by fuel type: This dataset shows the energy consumption of different countries categorized by fossil fuels (coal, oil, gas), renewables, and other sources.
* Population by country: This dataset provides population data for different countries over time.

By combining these datasets, you can calculate CO2 emissions per capita and analyze the relationship between various factors like energy consumption, population, and CO2 emissions.

**To identify the countries making the biggest strides in decreasing CO2 output, you can use the following datasets:**

* CO2 emissions by country: This dataset will be useful again to track changes in CO2 emissions over time.
* Annual change in CO2 emissions: This dataset provides the annual percentage change in CO2 emissions for different countries.
* Renewable energy production: This dataset includes information on the production of renewable energy, such as solar, wind, hydro, and geothermal.

By analyzing the annual change in CO2 emissions and comparing it with the growth in renewable energy production, you can identify countries that are successfully reducing their CO2 output.

**For determining the non-fossil fuel energy technology with the best price in the future, you can consider the following datasets:**

* Costs of different energy technologies: This dataset provides information on the costs of various energy technologies, including solar, wind, hydro, nuclear, and others. It can help you analyze the trends and projections for future energy costs.
* Renewable energy capacity and investment: This dataset shows the installed capacity and investment trends in renewable energy sources, which can provide insights into the growth potential and competitiveness of different technologies.

By examining the cost trends and comparing the capacity and investment data, you can assess which non-fossil fuel energy technology is expected to have the most favorable price in the future.


**Find what countries do:**

* did they make any pledges
* what kind of energy are they using now


* What contributes to CO2 output?

* What are the prices for the different types of non-fossil energy?

* What happens when a country outsources everything?

* How about CO2 vs the other GHG (greenhouse gasses)? Are there countries where CO2 falls but the other rise?

# 1: Biggest predictor of CO2 output

# 2: Biggest strides in decreasing CO2 output

First let's look at what countries produces, import, export and how that changed through the years.

* **co2**: annual total production-based co2 emission. measured in million tonnes. Measured in million tonnes.
* **co2_growth_abs**: annual growth of production-based co2 emission. Measured in million tonnes.
* **co2_growth_prct**: annual percentage growth of production-based co2 emission. measured in million tonnes. Measured in million tonnes.
* **co2_per_capita**, annual total production-based emissions of carbon dioxide (CO₂) per capita, measured in tonnes per person. 

* **trade_co2**: annual net carbon dioxide (CO₂) emissions embedded in trade. The net of import or export via traded goods. Positive = importer of CO₂ emissions; Negative =  exporter. Measured in million tonnes.
* **consumption_co2**: total co2 minus emissions embedded in exports, plus emissions embedded in imports. Consumption > production = importer of CO₂ emissions; Consumption < Production =  exporter. Measured in million tonnes.
* **consumption_co2**: total co2 minus emissions embedded in exports per capita, plus emissions embedded in imports. Consumption > production = importer of CO₂ emissions; Consumption < Production =  exporter. Measured in tonnes per person.

Note for later: 
also interesting: 
* what do we see when we look at rich vs poor (columns regarding GDP)
* what do we see when we look at the CO2 per energy unit.